# Molecular time evolution resource counts

Computes the resources to do Trotterized time evolution, a subroutine used in quantum Krylov and quantum phase estimation, of molecular Hamiltonians on IBM Quantum computers.

## Setup

In [1]:
import pickle

import matplotlib.pyplot as plt
import numpy as np
import scipy

import openfermion as of
from openfermionpyscf import run_pyscf
from openfermion.chem import geometry_from_pubchem, MolecularData


import qiskit
import qiskit.qasm3
import qiskit_ibm_runtime
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.synthesis import LieTrotter, SuzukiTrotter

from pytket.extensions.qiskit import IBMQBackend, tk_to_qiskit, qiskit_to_tk

import convert
import openfermion_helper

## Set Hamiltonian

In [2]:
# Water molecule.
# geometry = geometry_from_pubchem("water")
# basis = "sto-3g"
# multiplicity = 1
# charge = 0

# # Make molecule.
# molecule = MolecularData(geometry, basis, multiplicity, charge)

# mol = run_pyscf(molecule, run_mp2=True, run_cisd=True, run_ccsd=True, run_fci=True)
# mol.save()

In [3]:
# One water phosphate.
geometry = [
    ("P", (-1.034220, -0.234256,0.672434)),
    ("O", (-1.004065, 0.890081, -0.334695)),
    ("O", (-0.003166, -1.329504, 0.557597)),
    ("O", (-2.065823, -0.232403, 1.765329)),
    ("H", (0.881055, 0.866924, -1.063283)),
    ("O", (1.748944, 0.417505, -1.047631)),
    ("H", (1.477276, -0.378346, -0.549750)),
]
basis = "sto-3g"
multiplicity = 1
charge = 1

molecule = MolecularData(geometry, basis, multiplicity, charge)
mol = run_pyscf(molecule, run_mp2=True, run_cisd=False, run_ccsd=False, run_fci=False)
mol.save()

In [4]:
hamiltonian = MolecularData(filename=molecule.filename)
hamiltonian = hamiltonian.get_molecular_hamiltonian()
hamiltonian = of.get_fermion_operator(hamiltonian)
print(f"Fermionic Hamiltonian has {len(hamiltonian.terms)} term(s).")

Fermionic Hamiltonian has 3577527 term(s).


In [5]:
threshold = 1e-2
hamiltonian.compress(abs_tol=threshold)
print(f"Compressed Fermionic Hamiltonian has {len(hamiltonian.terms)} term(s).")

Compressed Fermionic Hamiltonian has 53211 term(s).


In [6]:
hamiltonian_openfermion = of.jordan_wigner(hamiltonian)

nqubits = openfermion_helper.get_num_qubits(hamiltonian_openfermion)
nterms = len(hamiltonian_openfermion.terms)

print(f"Qubit Hamiltonian acts on {nqubits} qubit(s) and has {nterms} term(s).")

Qubit Hamiltonian acts on 62 qubit(s) and has 24078 term(s).


In [7]:
hamiltonian = openfermion_helper.preprocess_hamiltonian(
    hamiltonian_openfermion,
    drop_term_if=[lambda term: term == ()],
    verbose=True,
)  # Drop identity and convert to Cirq PauliSum.

nterms = len(hamiltonian)
nqubits = len(hamiltonian.qubits)

print(f"\n\nPre-processed qubit Hamiltonian acts on {nqubits} qubit(s) and has {nterms} term(s).")

Status: On term 24077 = ((60, 'Z'), (61, 'Z')) (60, 'Y'), (61, 'X')) (59, 'Z'), (61, 'X')) (60, 'X'), (61, 'Z')) (57, 'Z'), (58, 'Z'), (59, 'Z'), (61, 'X')) (60, 'X'), (61, 'Z')) (59, 'X'), (61, 'Z')) (60, 'Y'), (61, 'X')) (59, 'X'), (61, 'Z')) (60, 'Y'), (61, 'X')) (60, 'X'), (61, 'Z')) (60, 'X'), (61, 'Z')) (60, 'X'), (61, 'Z')) (60, 'Z'), (61, 'X')) (56, 'Z'), (57, 'X')) (57, 'Z'), (58, 'Z'), (59, 'X'), (61, 'Z')) (60, 'Z'), (61, 'X'))(56, 'Z'), (57, 'X')) (56, 'Z'), (57, 'X')) (56, 'Z'), (57, 'X')) (58, 'Z'), (59, 'X')) (58, 'Z'), (59, 'X'))56, 'Z'), (57, 'Z'), (58, 'Z'), (59, 'X')), 'Z'), (57, 'X'))56, 'Z'), (57, 'X'))(59, 'Z'), (60, 'Z'), (61, 'X'))

Pre-processed qubit Hamiltonian acts on 62 qubit(s) and has 24077 term(s).


## Gate counts for a first order Trotter step

### (1) Crude estimate

In [8]:
# """Crude estimate."""
# num_cnots_crude: int = 0
# for term in hamiltonian:
#     num_cnots_crude += 2 ** (len(term.qubits) - 1)

# num_cnots_crude

### (2) Grouping + CNOT ladder

In [9]:
# """Estimate using grouping + diagonaliztion + exp(Z...Z) "ladder"."""
# import kcommute


# groups = kcommute.get_si_sets(hamiltonian, k=nqubits)

# num_cnots: int = 0
# for group in groups:
#     num_cnots += nqubits ** 2  # It takes O(n^2) Clifford gates to diagonalize all terms in this group [https://arxiv.org/abs/quant-ph/0406196].
#     for term in group:
#         num_cnots += 2 * len(term.qubits)  # Using 2w CNOTs in a "ladder" and one exp(Z) gate on the bottom qubit. See https://arxiv.org/abs/2408.08265v3 Fig. 3.
#     num_cnots += nqubits ** 2  # Rotating back to the Z basis (undoing the diagonal unitary).

# num_cnots

### (3) Qiskit's `PauliHedral` method

In [10]:
H = convert.cirq_pauli_sum_to_qiskit_pauli_op(hamiltonian)

In [11]:
# Following https://qiskit-community.github.io/qiskit-algorithms/tutorials/13_trotterQRTE.html.
order: int = 1
cx_structure = "chain"  # "fountain"
trotter_step = PauliEvolutionGate(H, time=1, synthesis=LieTrotter(cx_structure=cx_structure) if order == 1 else SuzukiTrotter(order, cx_structure=cx_structure))

circuit = qiskit.QuantumCircuit(H.num_qubits)
circuit.append(trotter_step, range(H.num_qubits));

circuit = circuit.decompose(reps=2)

print(
    f"""
Depth: {circuit.depth()}
Gates: {", ".join([f"{k.upper()}: {v}" for k, v in circuit.count_ops().items()])}
"""
)
# circuit.draw(fold=-1)


Depth: 774786
Gates: CX: 673806, H: 67104, SDG: 67072, S: 67072, U2: 67072, U1: 22170, RZ: 1907, RX: 32



## Optimize the circuit

In [12]:
compiled = qiskit.transpile(
    circuit,
    optimization_level=3,
    basis_gates=["u3", "cx"]
)
print(
    f"""
Depth: {compiled.depth()}
Gates: {", ".join([f"{k.upper()}: {v}" for k, v in compiled.count_ops().items()])}
"""
)
# compiled.draw(fold=-1)


Depth: 647787
Gates: CX: 593475, U3: 131158



## Compile to an IBM quantum computer

In [13]:
computer = qiskit_ibm_runtime.fake_provider.FakeKyiv()

compiled_kyiv = qiskit.transpile(
    compiled,
    backend=computer,
    optimization_level=3,
)
print(
    f"""
Depth: {compiled_kyiv.depth()}
Gates: {", ".join([f"{k.upper()}: {v}" for k, v in compiled_kyiv.count_ops().items()])}
"""
)


Depth: 4197360
Gates: RZ: 3820020, SX: 2320979, ECR: 1108679, X: 155630



: 

### Using Quantinuum's TKET compiler

In [ ]:
ibmbackend = IBMQBackend(backend_name="ibm_kyiv")  # Assumes a saved account. 

tket_compiled = ibmbackend.get_compiled_circuit(
    circuit=qiskit_to_tk(circuit),
    optimisation_level=3,
)
compiled_kyiv_tket = tk_to_qiskit(tket_compiled)

print(
    f"""
Depth: {compiled_kyiv_tket.depth()}
Gates: {", ".join([f"{k.upper()}: {v}" for k, v in compiled_kyiv_tket.count_ops().items()])}
"""
)

## Compute the number of Trotter steps needed for chemical accuracy

The Trotter error $\epsilon$ decreases linearly in the number of Trotter stops $r$. Using error bounds for $\epsilon$ from https://arxiv.org/abs/1912.08854 and chemical accuracy $\epsilon^* = 10^{-3}$ Ha, we estimate the number of Trotter steps needed for chemical accuracy as $r = \epsilon / \epsilon^*$.

In [ ]:
error = np.sum(np.abs(H.coeffs))  # Loose error bound from https://arxiv.org/abs/1912.08854.

epsilon: float = 0.001  # mHa

nsteps = round(error / epsilon)
nsteps

## VQE resources

### Gates

Read in VQE circuits.

In [ ]:
circuit_vqe = qiskit.qasm3.loads(pickle.load(open("kyiv_circuit_h2o", "rb")))

print(
    f"""
              Depth: {circuit_vqe.depth()}
         Gate count: {len(circuit_vqe)}
Nonlocal gate count: {circuit_vqe.num_nonlocal_gates()}
     Gate breakdown: {", ".join([f"{k.upper()}: {v}" for k, v in circuit_vqe.count_ops().items()])}
"""
)

## Number of shots

We compute the number of shots needed to compute one energy (cost function) $\langle \psi | H | \psi \rangle$ to accuracy $\epsilon$, assuming an (approximate) ground state $|\psi\rangle$.

In [ ]:
eval, evec = scipy.sparse.linalg.eigsh(
    of.linalg.get_sparse_operator(hamiltonian_openfermion),
    k=1,
    which="SA"
)
evec = evec.flatten()

In [22]:
import kcommute

In [ ]:
epsilons = [0.1, 0.01, 0.001]
kvals = [1, 2, nqubits // 2, nqubits]

all_shots = []
for k in kvals:
    groups = kcommute.get_si_sets(hamiltonian, k=k)
    groups_of = convert.to_groups_of(groups)
    base_shots = kcommute.compute_shots(groups_of, evec, epsilon=1)
    shots = [base_shots / epsilon ** 2 for epsilon in epsilons]
    all_shots.append(shots)
    print(all_shots)

In [ ]:
plt.rcParams.update({"font.family": "serif", "font.size": 12})


for kval, shots in zip(kvals, all_shots):
    plt.loglog(epsilons, shots, "--o", alpha=0.75, mec="black", label=f"$k = {kval}$")

plt.legend()
plt.xlabel("Accuracy $\epsilon$")
plt.ylabel("Shots $N$")
plt.title(f"$k$-commuting shot counts w.r.t. ground state\n for {mol.name} ({nqubits} qubits, {nterms} Paulis)");